# Putting the BP Historical Dataset into a SQLite Database

This notebook follows an exploration of the BP Historical dataset in [analyze-bp-historical-data/notebooks/explore.ipynb](https://github.com/ADGEfficiency/analyze-bp-historical-data/blob/main/notebooks/explore.ipynb).

## Options for Interacting with SQLite

There are a number of ways to interact with SQLite in Python:

1. the `sqlite3` module from the Python standard library,
2. through the `sqlalchemy` Python module (third party),
3. through `pandas` and a little bit of `sqlalchemy`.

We will use the third way with `pandas`.

## Create SQLite Table with `pandas`

First let's load our raw data into a `pd.DataFrame`:

In [1]:
import pandas as pd

raw = pd.read_csv("https://raw.githubusercontent.com/ADGEfficiency/analyze-bp-historical-data/main/data/bp-stats-review-2022-consolidated-dataset-narrow-format.csv")
raw.shape

(277095, 12)

Now we have our data, we need to next create a connection to our database using `sqlalchemy.create_engine`:

In [2]:
from sqlalchemy import create_engine

connection = create_engine('sqlite:///../data/db.sqlite', echo=True)

Finally, we can bring `pandas` and `sqlalchemy` together by using the connection in `pd.DataFrame.to_sql`:

In [3]:
raw.to_sql('raw', if_exists='replace', con=connection)

2023-02-20 14:41:29,697 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw")
2023-02-20 14:41:29,698 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:29,700 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("raw")
2023-02-20 14:41:29,701 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:29,703 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-02-20 14:41:29,704 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:29,705 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("raw")
2023-02-20 14:41:29,706 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:29,707 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-02-20 14:41:29,708 INFO sqlalchemy.engine.Engine [raw sql] ('raw',)
2023-02-20 14:41:29,710 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("raw")
2023-02-20 14:41:2

277095

We use `if_exists='replace'` to completely replace the table each time we run `.to_sql`.

And that's it!  Pretty simple if you know the simple ways ^^

## Read Entire SQLite Table into `pandas`

Now we have a SQLite database - it's a file in the `data` folder:

In [4]:
!ls ./data

ls: ./data: No such file or directory


We can read data from the database back into `pandas` by using `pd.read_sql`:

In [5]:
out = pd.read_sql("SELECT * FROM raw", con=connection)
print(out.shape)
out.head(3)

2023-02-20 14:41:33,161 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM raw")
2023-02-20 14:41:33,162 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:33,162 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM raw")
2023-02-20 14:41:33,163 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:33,164 INFO sqlalchemy.engine.Engine SELECT * FROM raw
2023-02-20 14:41:33,165 INFO sqlalchemy.engine.Engine [raw sql] ()
(277095, 13)


,index,Country,Year,ISO3166_alpha3,ISO3166_numeric,Region,SubRegion,OPEC,EU,OECD,CIS,Var,Value
0,0,Brazil,2004,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,0.000000
1,1,Brazil,2005,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,0.014737
2,2,Brazil,2006,BRA,76.0,S. & Cent. America,South America,0.0,0.0,0.0,0.0,biodiesel_cons_kboed,1.016887


We use a SQL statement `SELECT * FROM raw`.  This statement selects the entire `raw` table.

## Filter SQLite Table

We can also do analytics on the database directly.

Previously in [analyze-bp-historical-data/notebooks/explore.ipynb](https://github.com/ADGEfficiency/analyze-bp-historical-data/blob/main/notebooks/explore.ipynb) we manipulated data using pandas.

It's also possible to do analytics with the database.  When your database is a remote server (say Postgres), pushing computation back into the database can reduce the load on your local machine.  

Below we use a `SELECT` statement with a `WHERE` clause to get only wind data from New Zealand:

In [6]:
out = pd.read_sql("SELECT * FROM raw WHERE Country = 'New Zealand' AND Var like '%wind%'", con=connection)
out.head(3)

2023-02-20 14:41:34,659 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM raw WHERE Country = 'New Zealand' AND Var like '%wind%'")
2023-02-20 14:41:34,660 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:34,661 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM raw WHERE Country = 'New Zealand' AND Var like '%wind%'")
2023-02-20 14:41:34,662 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-20 14:41:34,665 INFO sqlalchemy.engine.Engine SELECT * FROM raw WHERE Country = 'New Zealand' AND Var like '%wind%'
2023-02-20 14:41:34,665 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,Country,Year,ISO3166_alpha3,ISO3166_numeric,Region,SubRegion,OPEC,EU,OECD,CIS,Var,Value
0,264879,New Zealand,1974,NZL,554.0,Asia Pacific,Asia Pacific,0.0,0.0,1.0,0.0,wind_ej,0.0
1,264880,New Zealand,1975,NZL,554.0,Asia Pacific,Asia Pacific,0.0,0.0,1.0,0.0,wind_ej,0.0
2,264881,New Zealand,1976,NZL,554.0,Asia Pacific,Asia Pacific,0.0,0.0,1.0,0.0,wind_ej,0.0


Compare how short and sweet this is - SQL at it's best.

## Summary

Key takeaways:

- we can interact with SQLite from Python in a few ways,
- `pd.to_sql` is a simple way to create a `SQLite` table from a CSV,
- we can select all data using a `SELECT *` SQL query,
- we can filter data using a `SELECT * WHERE` SQL query.